In [1]:
import cobra
import scipy
import os
from pathlib import Path

In [2]:
# create dictionary for all models
models = {}

# load human GEM
models["ihuman"] = cobra.io.load_matlab_model("./models/Human-GEM.mat")

In [ ]:

# load ftINIT models
directory = os.fsencode("./models")
    
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if filename.endswith("_model.mat"): 
        with open("./models/"+filename) as model:
            pos = filename.find("_")
            models[(filename[:pos])] = cobra.io.load_matlab_model(model)
    else:
        continue

print(models)

UnicodeDecodeError: 'charmap' codec can't decode byte 0x90 in position 181: character maps to <undefined>

In [3]:
# load model generated using example data from ftINIT paper
test = cobra.io.load_matlab_model("./models/test_model.mat")

In [4]:
# load tissue specific model
skcm = cobra.io.load_matlab_model("./models/SKCM_model.mat")

In [10]:
print("iHuman:")
print(f"Genes: {len(ihuman.genes)}")
print(f"Reactions: {len(ihuman.reactions)}")
print(f"Groups: {len(ihuman.groups)}")
print(f"Growth: {ihuman.optimize()}")
print()
print("gtex-Model:")
print(f"Genes: {len(test.genes)}")
print(f"Reactions: {len(test.reactions)}")
print(f"Groups: {len(test.groups)}")
print(f"Growth: {test.optimize()}")
print()
print("SKCM:")
print(f"Genes: {len(skcm.genes)}")
print(f"Reactions: {len(skcm.reactions)}")
print(f"Groups: {len(skcm.groups)}")
print(f"Growth: {skcm.optimize()}")

iHuman:
Genes: 2889
Reactions: 12995
Groups: 148
Growth: <Solution 124.868 at 0x27df4f57440>

gtex-Model:
Genes: 2415
Reactions: 8266
Groups: 136
Growth: <Solution 124.722 at 0x27df4f57980>

SKCM:
Genes: 2415
Reactions: 8340
Groups: 138
Growth: <Solution 124.804 at 0x27df4f54560>


In [15]:
deleted_groups = []
for group in ihuman.groups:
    if group not in skcm.groups:
        deleted_groups.append(group.name)
        print(group.name)

	Arachidonic acid metabolism
Alkaloids biosynthesis
Fatty acid degradation
Hippurate metabolism
Insect hormone biosynthesis
Peptide metabolism
Toluene degradation
Triacylglycerol synthesis
Ubiquinone and other terpenoid-quinone biosynthesis
octane oxidation
